In [ ]:
#required imports and dependencies

import pandas as pd
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")
from sklearn import preprocessing
from keras import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#our own confusion matrix module that we developed
import cmatrix as cm

#load data into data frames
xlsPath = r'C:\Users\New\Desktop\UniWork\ADA\ada2\final dataset.xlsx'
data = pd.read_excel(xlsPath)

#split into data and class variable
X = data.drop(['date','symbol','price increase tomorrow?'], axis =1)

#perform normalization on data 
min_max_scaler = preprocessing.MinMaxScaler()
for column in X:
    X[column] = min_max_scaler.fit_transform(X[column].values.reshape(-1,1))
class_var = data['price increase tomorrow?']

#select dimensions to reduce to 
inputDims = 4
attributes = SelectKBest(chi2, k=inputDims).fit_transform(X,class_var)

#Create a train test split
split_number = 1406
train_attributes,train_class_var,test_attributes,test_class_var = \
attributes[:split_number],class_var[:split_number], \
attributes[split_number:],class_var[split_number:]



In [ ]:
def sequentialNN(train_attributes,train_class_var,test_attributes,test_class_var):
    
    model = Sequential()
    # The imput dim here is the number of cols in the df getting fed into the model
    model.add(Dense(256, input_dim=inputDims, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])


    model.summary()

    model.fit(train_attributes, train_class_var,
             epochs=100, verbose=0, batch_size=1406)

    #the predict_classes method returns the binary variable we're looking for
    # for some reason it doesn't output the correct array type, made a 2D array
    pred = model.predict_classes(test_attributes)

    #the flatten method solves this and squashes to a 1D array for evaluation
    predictions = pred.flatten()

    #be sure to import our cmatrix module
    cmat = cm.cmatrix(test_class_var, predictions)
    return cmat





In [ ]:
#Initialize our model and print confusion matrix

seqNN=sequentialNN(train_attributes,train_class_var,test_attributes,test_class_var)
seqNN
    